<a href="https://colab.research.google.com/github/sushantagarwal29/ragpoc/blob/main/CaseStudy3-usingDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index
%pip install kdbai-client
%pip install llama-index-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.2/790.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 6.1 MB/s eta 0:00:00


In [2]:
%pip install llama-index-node-parser-topic

In [3]:
%pip install -U llama-index-llms-azure-inference
%pip install -U llama-index-embeddings-azure-inference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 7.7 MB/s eta 0:00:00


In [4]:
from llama_index.llms.azure_inference import AzureAICompletionsModel
from llama_index.embeddings.azure_inference import AzureAIEmbeddingsModel

In [5]:
from llama_index.core import Settings

In [ ]:
import os
from getpass import getpass
# Set up environment variables for API keys and endpoints
os.environ["LLAMA_CLOUD_API_KEY"] = (
    os.environ["LLAMA_CLOUD_API_KEY"]
    if "LLAMA_CLOUD_API_KEY" in os.environ
    else getpass("LLAMA CLOUD API key: ")
)

AZURE_OPENAI_ENDPOINT = (
    os.environ["AZURE_OPENAI_ENDPOINT"]
    if "AZURE_OPENAI_ENDPOINT" in os.environ
    else input("AZURE_OPENAI_ENDPOINT: ")
)
AZURE_OPENAI_API_KEY = (
    os.environ["AZURE_OPENAI_API_KEY"]
    if "AZURE_OPENAI_API_KEY" in os.environ
    else getpass("AZURE_OPENAI_API_KEY: ")
)

# Initialize the LLM and embedding models
llm = AzureAICompletionsModel(
    endpoint=AZURE_OPENAI_ENDPOINT + "/openai/deployments/gpt-4o",
    credential=AZURE_OPENAI_API_KEY,
    api_version="2024-08-01-preview",
)

embed_model = AzureAIEmbeddingsModel(
    endpoint=AZURE_OPENAI_ENDPOINT + "/openai/deployments/text-embedding-3-small",
    credential=AZURE_OPENAI_API_KEY,
    model_name="text-embedding-3-small",
)

# Set the LLM and embedding model in the Settings object
Settings.llm = llm
Settings.embed_model = embed_model

AZURE_OPENAI_ENDPOINT: https://ai-depoc1aihub1643128651037.openai.azure.com/
AZURE_OPENAI_API_KEY: ··········


In [ ]:
from llama_index.node_parser.topic import TopicNodeParser

In [ ]:
node_parser = TopicNodeParser.from_defaults(
    llm=llm,
    max_chunk_size=1000,
    similarity_method="llm",  # can be "llm" or "embedding"
    window_size=2,  # paper suggests window_size=5
)

In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["/content/Daily_Sports_Highlights_01092024_With_Subsections.pdf"]).load_data()

In [ ]:
print(documents[0].get_content())

01.09.2024
Daily Sports Highlights
Football
Lionel Messi Leads Inter Miami to Victory in MLS Cup Final
Source: ESPN - September 1, 2024
Lionel Messi's incredible performance secured Inter Miami's first MLS Cup victory, defeating LAFC 3-2 in a thrilling final.
Messi scored twice, with his second goal in the 85th minute sealing the win. This victory cements Messi's status as one
of the greatest footballers in the history of the sport.
Link: https://www.espn.com/soccer/story/lionel-messi-leads-inter-miami-victory-mls-cup-final
Cristiano Ronaldo Scores 800th Career Goal
Source: Sky Sports - September 1, 2024
Cristiano Ronaldo has reached another milestone in his illustrious career by scoring his 800th career goal. The
Portuguese star continues to break records and is widely regarded as one of the greatest footballers of all time.
Link: https://www.skysports.com/football/ronaldo-scores-800th-goal
Tennis
Serena Williams Announces Retirement After US Open
Source: BBC Sports - September 1, 202

In [ ]:
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(nodes[3].get_content())

Cristiano Ronaldo scored his 800th career goal.
